In [1]:
import json
# import evaluate

from typing import Any, Dict, List, Tuple
from tqdm import tqdm

import nlp2go

In [2]:
import pandas as pd

In [3]:
import evaluate

In [4]:
# metrics:
bleu = evaluate.load("bleu")
sbleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /home/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/user/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
help(bleu.compute)

Help on method compute in module evaluate.module:

compute(*, predictions=None, references=None, **kwargs) method of evaluate_modules.metrics.evaluate-metric--bleu.9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76.bleu.Bleu instance
    Compute the evaluation module.
    
            Usage of positional arguments is not allowed to prevent mistakes.
    
            Args:
                predictions (`list/array/tensor`, *optional*):
                    Predictions.
                references (`list/array/tensor`, *optional*):
                    References.
                **kwargs (optional):
                    Keyword arguments that will be forwarded to the evaluation module [`~evaluate.EvaluationModule.compute`]
                    method (see details in the docstring).
    
            Return:
                `dict` or `None`
    
                - Dictionary with the results if this evaluation module is run on the main process (`process_id == 0`).
                -

In [6]:
bdg_model = nlp2go.Model('BDG.pt')
bdg_pm_model = nlp2go.Model('BDG_PM.pt')
bdg_anpm_model = nlp2go.Model('BDG_ANPM.pt')

/home/user/.pyenv/versions/3.8.20/lib/python3.8/site-packages/tfkit/utility/model.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torchpack = torch.load(model_path, map

===model info===
model_config bert-base-cased
tags ['onebyone_0']
type ['onebyone']
maxlen 512
epoch 6
loading saved model


/home/user/.pyenv/versions/3.8.20/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


===ADD TOKEN===
We have added 0 tokens
Using device: cuda
finish loading
loaded model predict_parameter {}
===model info===
model_config bert-base-cased
tags ['onebyone-pos_0']
type ['onebyone-pos']
maxlen 512
epoch 6
loading saved model
===ADD TOKEN===
We have added 0 tokens
Using device: cuda
finish loading
loaded model predict_parameter {}
===model info===
model_config bert-base-cased
tags ['onebyone-both_0']
type ['onebyone-both']
maxlen 512
epoch 6
loading saved model
===ADD TOKEN===
We have added 0 tokens
Using device: cuda
finish loading
loaded model predict_parameter {}


In [7]:
type(bdg_model)

nlp2go.model.Model

In [8]:
input_json = {
    "input": """When you ' re having a holiday , one of the main questions to ask is which hotel or apartment to choose . However , when it comes to France , you have another special choice : treehouses . In France , treehouses are offered to travelers as a new choice in many places . The price may be a little higher , but you do have a chance to _ your childhood memories . Alain Laurens , one of France ' s top treehouse designers , said , " Most of the people might have the experience of building a den when they were young . And they like that feeling of freedom when they are children . " Its fairy - tale style gives travelers a special feeling . It seems as if they are living as a forest king and enjoying the fresh air in the morning . Another kind of treehouse is the " star cube " . It gives travelers the chance of looking at the stars shining in the sky when they are going to sleep . Each " star cube " not only offers all the comfortable things that a hotel provides for travelers , but also gives them a chance to look for stars by using a telescope . The glass roof allows you to look at the stars from your bed . [SEP] The passage mainly tells us [SEP] treehouses in france."""
}

In [9]:
bdg_pm_model.predict(input_json, decodenum=3)

{'result': ['a new design in different countries.',
  'the design of a special house.',
  'special ways of finding different houses.']}

In [10]:
with open("tf_dataset_pretty_filtered.json", 'r', encoding="utf8") as inp:
    data_tf = json.load(inp)

with open("title_dataset_pretty_filtered.json", 'r', encoding="utf8") as inp:
    data_title = json.load(inp)

In [11]:
data_tf["val"][0]

{'example_id': 'high12024.txt',
 'article': '"Is there anything else you need, honey?" my dad asked me as he put three twenty dollar bills in my hand. I was traveling back home from a family visit, and after treating me to breakfast and filling my car with gas, it was obvious that my dad wanted to make sure that I would be okay on the road.\n"No, Dad. You\'ve done so much already. Thank you!" I was overwhelmed once again by his kind acts of providing everything I needed, although I turned 40. Yet I realize that in my father\'s eyes, I will always be his little girl. He takes deep pleasure in knowing his children are all right. Now that he has enough money, he loves to give whenever he sees a need.\nBut this was not always the case. Divorced from my mother when I was 11, my dad couldn\'t be around his kids as often as he would have liked. Money was also tight; even weekend visits were rare. However, my dad stayed in constant communication with us and made sure he was involved in our liv

In [12]:
ANSWER_ID_DICT = {"A": 0, "B": 1, "C": 2, "D": 3}

def create_model_input(example: Dict[str, Any]) -> str:
    right_answer = example["options"][ANSWER_ID_DICT[example["answer"]]]
    output = f"{example['article']} [SEP] {example['question']} [SEP] {right_answer}"
    return output

In [13]:
def get_ground_truth_distractors(example: Dict[str, Any]) -> str:
    right_answer = example["options"][ANSWER_ID_DICT[example["answer"]]]
    options = [
        '; '.join([f'"{option}"' for option in example["options"] if option != right_answer])
    ]
    return options

In [14]:
def get_model_output(example: Dict[str, Any], model: nlp2go.model.Model) -> str:
    model_input = create_model_input(example)
    predicted = model.predict(model_input, decodenum=3)["result"]
    model_output = ['; '.join([f'"{option}"' for option in predicted])]
    return model_output

In [15]:
def get_metric_inputs_seq2seq(example: Dict[str, Any], model: nlp2go.model.Model) -> Tuple[str, str]:
    distractors = get_ground_truth_distractors(example)
    output = get_model_output(example, model)
    return distractors, output

In [16]:
def compute_metrics(output: List[str], label_batch: List[str]) -> dict:
    metric_dict = {
        "bleu": bleu.compute(predictions=output, references=[[label] for label in label_batch], max_order=4),
        "sbleu": sbleu.compute(predictions=output, references=[[label] for label in label_batch]),
        "rouge": rouge.compute(predictions=output, references=label_batch),
        "meteor": meteor.compute(predictions=output, references=label_batch)
    }
    return metric_dict

In [17]:
model_metrics_tf_val = dict()

for model_name, model in zip(
    ["bdg", "bdgpm", "bdganpm"],
    [bdg_model, bdg_pm_model, bdg_anpm_model]
):
    model_metrics_tf_val[model_name] = []
    for item in tqdm(data_tf["val"]):
        right_answer = item["options"][ANSWER_ID_DICT[item["answer"]]]
        distractors, output = get_metric_inputs_seq2seq(item, model)
        metric = compute_metrics(output, [distractors])
        model_metrics_tf_val[model_name].append(
            {
                "article": item["article"],
                "right_answer": right_answer,
                "question": item["question"],
                "distractors": distractors,
                "output": output,
    
                "bleu": metric["bleu"]["bleu"],
                "sbleu": metric["sbleu"]["score"],
                "rouge1": metric["rouge"]["rouge1"],
                "rouge2": metric["rouge"]["rouge2"],
                "rougeL": metric["rouge"]["rougeL"],
                "rougeLsum": metric["rouge"]["rougeLsum"],
                "meteor": metric["meteor"]["meteor"]
            }
        )
    model_metrics_tf_val[model_name] = pd.DataFrame(
        model_metrics_tf_val[model_name]
    )

100%|██████████| 175/175 [03:44<00:00,  1.28s/it]


In [18]:
model_metrics_tf_test = dict()

for model_name, model in zip(
    ["bdg", "bdgpm", "bdganpm"],
    [bdg_model, bdg_pm_model, bdg_anpm_model]
):
    model_metrics_tf_test[model_name] = []
    for item in tqdm(data_tf["test"]):
        right_answer = item["options"][ANSWER_ID_DICT[item["answer"]]]
        distractors, output = get_metric_inputs_seq2seq(item, model)
        metric = compute_metrics(output, [distractors])
        model_metrics_tf_test[model_name].append(
            {
                "article": item["article"],
                "right_answer": right_answer,
                "question": item["question"],
                "distractors": distractors,
                "output": output,
    
                "bleu": metric["bleu"]["bleu"],
                "sbleu": metric["sbleu"]["score"],
                "rouge1": metric["rouge"]["rouge1"],
                "rouge2": metric["rouge"]["rouge2"],
                "rougeL": metric["rouge"]["rougeL"],
                "rougeLsum": metric["rouge"]["rougeLsum"],
                "meteor": metric["meteor"]["meteor"]
            }
        )
    model_metrics_tf_test[model_name] = pd.DataFrame(
        model_metrics_tf_test[model_name]
    )

100%|██████████| 187/187 [04:05<00:00,  1.32s/it]


In [19]:
model_metrics_title_val = dict()

for model_name, model in zip(
    ["bdg", "bdgpm", "bdganpm"],
    [bdg_model, bdg_pm_model, bdg_anpm_model]
):
    model_metrics_title_val[model_name] = []
    for item in tqdm(data_tf["val"]):
        right_answer = item["options"][ANSWER_ID_DICT[item["answer"]]]
        distractors, output = get_metric_inputs_seq2seq(item, model)
        metric = compute_metrics(output, [distractors])
        model_metrics_title_val[model_name].append(
            {
                "article": item["article"],
                "right_answer": right_answer,
                "question": item["question"],
                "distractors": distractors,
                "output": output,
    
                "bleu": metric["bleu"]["bleu"],
                "sbleu": metric["sbleu"]["score"],
                "rouge1": metric["rouge"]["rouge1"],
                "rouge2": metric["rouge"]["rouge2"],
                "rougeL": metric["rouge"]["rougeL"],
                "rougeLsum": metric["rouge"]["rougeLsum"],
                "meteor": metric["meteor"]["meteor"]
            }
        )
    model_metrics_title_val[model_name] = pd.DataFrame(
        model_metrics_title_val[model_name]
    )

100%|██████████| 175/175 [03:45<00:00,  1.29s/it]


In [20]:
model_metrics_title_test = dict()

for model_name, model in zip(
    ["bdg", "bdgpm", "bdganpm"],
    [bdg_model, bdg_pm_model, bdg_anpm_model]
):
    model_metrics_title_test[model_name] = []
    for item in tqdm(data_tf["test"]):
        right_answer = item["options"][ANSWER_ID_DICT[item["answer"]]]
        distractors, output = get_metric_inputs_seq2seq(item, model)
        metric = compute_metrics(output, [distractors])
        model_metrics_title_test[model_name].append(
            {
                "article": item["article"],
                "right_answer": right_answer,
                "question": item["question"],
                "distractors": distractors,
                "output": output,
    
                "bleu": metric["bleu"]["bleu"],
                "sbleu": metric["sbleu"]["score"],
                "rouge1": metric["rouge"]["rouge1"],
                "rouge2": metric["rouge"]["rouge2"],
                "rougeL": metric["rouge"]["rougeL"],
                "rougeLsum": metric["rouge"]["rougeLsum"],
                "meteor": metric["meteor"]["meteor"]
            }
        )
    model_metrics_title_test[model_name] = pd.DataFrame(
        model_metrics_title_test[model_name]
    )

100%|██████████| 187/187 [04:06<00:00,  1.32s/it]


In [26]:
import os
os.mkdir("bdg_model_results")

In [27]:
new_dir = "bdg_model_results/title_test"
os.mkdir(new_dir)
for title, df in model_metrics_title_test.items():
    df.to_csv(f"{new_dir}/{title}.csv")

In [28]:
new_dir = "bdg_model_results/title_val"
os.mkdir(new_dir)
for title, df in model_metrics_title_val.items():
    df.to_csv(f"{new_dir}/{title}.csv")

In [29]:
new_dir = "bdg_model_results/tf_test"
os.mkdir(new_dir)
for title, df in model_metrics_tf_test.items():
    df.to_csv(f"{new_dir}/{title}.csv")

In [30]:
new_dir = "bdg_model_results/tf_val"
os.mkdir(new_dir)
for title, df in model_metrics_tf_val.items():
    df.to_csv(f"{new_dir}/{title}.csv")

In [33]:
model_metrics_tf_val["bdg"].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,0.193187,19.465021,0.312899,0.131918,0.237657,0.237657,0.425536
std,0.119038,11.702684,0.167806,0.143336,0.139246,0.139246,0.162934
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019417
25%,0.124688,12.468845,0.202564,0.029199,0.156288,0.156288,0.365422
50%,0.176954,17.695382,0.320000,0.095238,0.237288,0.237288,0.447952
75%,0.257328,25.732779,0.416667,0.189578,0.309859,0.309859,0.514055
max,0.585709,58.570914,0.843373,0.740741,0.843373,0.843373,0.810864


In [34]:
model_metrics_tf_val["bdgpm"].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,0.343251,34.367216,0.489099,0.334022,0.380237,0.380237,0.535257
std,0.196418,19.575524,0.236596,0.231573,0.199764,0.199764,0.215084
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019417
25%,0.176151,17.615111,0.318809,0.101959,0.242641,0.242641,0.423261
50%,0.348135,34.813456,0.549451,0.340426,0.386364,0.386364,0.553563
75%,0.492200,49.219979,0.666667,0.505319,0.530931,0.530931,0.686359
max,0.844802,84.480242,0.965517,0.928571,0.965517,0.965517,0.974934


In [35]:
model_metrics_tf_val["bdganpm"].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,0.350780,35.118268,0.498028,0.338467,0.372920,0.372920,0.543424
std,0.193223,19.256085,0.238949,0.225279,0.187864,0.187864,0.214705
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019417
25%,0.197795,19.779493,0.320513,0.136574,0.264481,0.264481,0.438535
50%,0.371037,37.103740,0.555556,0.350877,0.385965,0.385965,0.571021
75%,0.483107,48.310722,0.666667,0.513390,0.500000,0.500000,0.697285
max,0.848120,84.812049,1.000000,0.913043,0.807018,0.807018,0.963674


In [36]:
model_metrics_tf_test["bdg"].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000
mean,0.182350,18.276234,0.300599,0.110799,0.222837,0.222837,0.421694
std,0.099692,9.909152,0.147126,0.106735,0.112930,0.112930,0.155136
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017036
25%,0.129903,12.990302,0.217129,0.033908,0.169703,0.169703,0.365608
50%,0.177597,17.759699,0.303797,0.090909,0.222222,0.222222,0.447520
75%,0.223535,22.353455,0.396970,0.155844,0.283883,0.283883,0.513068
max,0.607856,60.785627,0.750000,0.600000,0.583333,0.583333,0.751080


In [37]:
model_metrics_tf_test["bdgpm"].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000
mean,0.329639,32.999529,0.469274,0.309506,0.346877,0.346877,0.530824
std,0.180218,17.962860,0.224518,0.207744,0.173712,0.173712,0.203419
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017036
25%,0.199437,19.943694,0.340238,0.128032,0.238644,0.238644,0.432342
50%,0.349320,34.931988,0.515152,0.333333,0.369231,0.369231,0.562770
75%,0.443332,44.333219,0.625000,0.436364,0.440000,0.440000,0.671293
max,0.758395,75.839485,0.912281,0.830189,0.909091,0.909091,0.924915


In [38]:
model_metrics_tf_test["bdganpm"].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000
mean,0.335832,33.620182,0.480622,0.320795,0.368248,0.368248,0.527549
std,0.180442,17.981925,0.228681,0.213260,0.181088,0.181088,0.201066
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017036
25%,0.187111,18.711121,0.333333,0.146125,0.250272,0.250272,0.430467
50%,0.365679,36.567885,0.536585,0.333333,0.388350,0.388350,0.547636
75%,0.465812,46.581156,0.649626,0.481481,0.482343,0.482343,0.663316
max,0.783456,78.345579,0.956522,0.865672,0.877193,0.877193,0.928033


In [39]:
model_metrics_title_val["bdg"].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,0.193187,19.465021,0.312899,0.131918,0.237657,0.237657,0.425536
std,0.119038,11.702684,0.167806,0.143336,0.139246,0.139246,0.162934
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019417
25%,0.124688,12.468845,0.202564,0.029199,0.156288,0.156288,0.365422
50%,0.176954,17.695382,0.320000,0.095238,0.237288,0.237288,0.447952
75%,0.257328,25.732779,0.416667,0.189578,0.309859,0.309859,0.514055
max,0.585709,58.570914,0.843373,0.740741,0.843373,0.843373,0.810864


In [40]:
model_metrics_title_val["bdgpm"].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,0.343251,34.367216,0.489099,0.334022,0.380237,0.380237,0.535257
std,0.196418,19.575524,0.236596,0.231573,0.199764,0.199764,0.215084
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019417
25%,0.176151,17.615111,0.318809,0.101959,0.242641,0.242641,0.423261
50%,0.348135,34.813456,0.549451,0.340426,0.386364,0.386364,0.553563
75%,0.492200,49.219979,0.666667,0.505319,0.530931,0.530931,0.686359
max,0.844802,84.480242,0.965517,0.928571,0.965517,0.965517,0.974934


In [41]:
model_metrics_title_val["bdganpm"].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,0.350780,35.118268,0.498028,0.338467,0.372920,0.372920,0.543424
std,0.193223,19.256085,0.238949,0.225279,0.187864,0.187864,0.214705
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019417
25%,0.197795,19.779493,0.320513,0.136574,0.264481,0.264481,0.438535
50%,0.371037,37.103740,0.555556,0.350877,0.385965,0.385965,0.571021
75%,0.483107,48.310722,0.666667,0.513390,0.500000,0.500000,0.697285
max,0.848120,84.812049,1.000000,0.913043,0.807018,0.807018,0.963674


In [42]:
model_metrics_title_test["bdg"].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000
mean,0.182350,18.276234,0.300599,0.110799,0.222837,0.222837,0.421694
std,0.099692,9.909152,0.147126,0.106735,0.112930,0.112930,0.155136
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017036
25%,0.129903,12.990302,0.217129,0.033908,0.169703,0.169703,0.365608
50%,0.177597,17.759699,0.303797,0.090909,0.222222,0.222222,0.447520
75%,0.223535,22.353455,0.396970,0.155844,0.283883,0.283883,0.513068
max,0.607856,60.785627,0.750000,0.600000,0.583333,0.583333,0.751080


In [43]:
model_metrics_title_test["bdgpm"].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000
mean,0.329639,32.999529,0.469274,0.309506,0.346877,0.346877,0.530824
std,0.180218,17.962860,0.224518,0.207744,0.173712,0.173712,0.203419
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017036
25%,0.199437,19.943694,0.340238,0.128032,0.238644,0.238644,0.432342
50%,0.349320,34.931988,0.515152,0.333333,0.369231,0.369231,0.562770
75%,0.443332,44.333219,0.625000,0.436364,0.440000,0.440000,0.671293
max,0.758395,75.839485,0.912281,0.830189,0.909091,0.909091,0.924915


In [44]:
model_metrics_title_val["bdganpm"].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,0.350780,35.118268,0.498028,0.338467,0.372920,0.372920,0.543424
std,0.193223,19.256085,0.238949,0.225279,0.187864,0.187864,0.214705
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019417
25%,0.197795,19.779493,0.320513,0.136574,0.264481,0.264481,0.438535
50%,0.371037,37.103740,0.555556,0.350877,0.385965,0.385965,0.571021
75%,0.483107,48.310722,0.666667,0.513390,0.500000,0.500000,0.697285
max,0.848120,84.812049,1.000000,0.913043,0.807018,0.807018,0.963674
